## combining individual coverage data into a mean-normalized coverage

## Plan:

+ for the PanelofNormals, create a coverage file for each PON bam and each chromosome
+ Pon bams should optimally not contain CNV or there should be a lot of them to reduce the std
+ put all the files into a big matrix and normalize coverages and produce an average coverage (+ std) for the exonic space
+ compare the tumor samples against that PONcoverage to get differences in CNV

### alternative approach
+ get all samples and remove the outliers from std

In [ ]:
# get the code
import sys
sys.path.append('../code')
from pon_coverage import make_PON_coverage

In [ ]:
# set the paths
home = '/Users/mahtin'
home = '/Users/martinscience'
testdata = f"{home}/Dropbox/Icke/Work/somVar/testdata"
tooldata = f"{home}/Dropbox/Icke/Work/somVar/tooldata"
shell_path = "../shell"
static_path = f"{home}/Dropbox/Icke/Work/static"
bed_path = f"{static_path}/bed_files/SureSelect/hg38"

cnvdata = f"{tooldata}/myCNVdata/"
bedCov_path = f"{cnvdata}/bedCov"
bedCov_path = f"{home}/mount/scratch/develop/PONcoverage/bedCov"
output_path = f"{cnvdata}/output/PONcoverage"

### get all the normal samples from the PON list into df for normalization and averaging

In [ ]:
config = {
    'sample_PON_path': bedCov_path,
    'output_path': 'output_path',
    'normCov':100,  
    'minCov': 20,   
    'max_mean_std': 20,
    'std_factor': 2,    
} 
sample_list = [f"{str(s).zfill(3)}_B" for s in range(45)]

df = make_PON_coverage("chr7", sample_list, config)

In [ ]:
df